In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
!pip install openpyxl

In [9]:
df = pd.read_excel('../Github depo/QC_encoded_data.xlsx')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6448 entries, 0 to 6447
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                6403 non-null   object 
 1   finish              6448 non-null   object 
 2   delivered_pcs_pack  4807 non-null   object 
 3   pack_weight_kg      2411 non-null   object 
 4   OK_NOK              6448 non-null   object 
 5   pcs_lenght_mm       4514 non-null   float64
 6   quantity_pcs        4514 non-null   object 
 7   overal_weight_kg    4488 non-null   float64
 8   pcs_pck             6350 non-null   float64
 9   name_enc            6448 non-null   object 
 10  name_e              6448 non-null   object 
 11  drawing_no_e        6448 non-null   object 
 12  shape_no_e          6448 non-null   object 
 13  supplier_e          6448 non-null   object 
dtypes: float64(3), object(11)
memory usage: 705.4+ KB


In [11]:
df

,date,finish,delivered_pcs_pack,pack_weight_kg,OK_NOK,pcs_lenght_mm,quantity_pcs,overal_weight_kg,pcs_pck,name_enc,name_e,drawing_no_e,shape_no_e,supplier_e
0,"13,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,A,Regor,DR0,SH0,SU0
1,"13,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,B,Procyon,DR1,SH1,SU0
2,"13,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,A,Regor,DR0,SH0,SU0
3,"13,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,A,Regor,DR0,SH0,SU0
4,"19,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,C,Grus,DR2,SH2,SU1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6443,2022-09-13 00:00:00,C-0,100,38.2,OK,4150.0,900,343.6200,100.0,FX,Voyager,DR111,SH82,SU1
6444,2022-09-13 00:00:00,C-0,100,38.2,OK,4150.0,920,351.2560,100.0,FX,Voyager,DR111,SH82,SU1
6445,2022-09-13 00:00:00,SUR,40,12.2,OK,4050.0,1590,482.9625,60.0,EI,Aludra,DR11,SH10,SU1
6446,2022-09-13 00:00:00,SUR,40,10.5,OK,4050.0,2032,534.9240,100.0,Y,Wasat,DR16,SH15,SU1


In [14]:
df.head()

,date,finish,delivered_pcs_pack,pack_weight_kg,OK_NOK,pcs_lenght_mm,quantity_pcs,overal_weight_kg,pcs_pck,name_enc,name_e,drawing_no_e,shape_no_e,supplier_e
0,"13,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,A,Regor,DR0,SH0,SU0
1,"13,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,B,Procyon,DR1,SH1,SU0
2,"13,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,A,Regor,DR0,SH0,SU0
3,"13,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,A,Regor,DR0,SH0,SU0
4,"19,01,2017",C-0,NaN,NaN,NOK,NaN,NaN,NaN,40.0,C,Grus,DR2,SH2,SU1
